In [1]:
DATA_PATH = "./data/"
OUTPUT_PATH = "./out/"

In [26]:
import os
import ast
import numpy as np
import pandas as pd
from datetime import datetime

#from pandarallel import pandarallel
#pandarallel.initialize()

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [52]:
lst = []
for filename in os.listdir(DATA_PATH):
    if filename.endswith(".csv") and not(filename.endswith("_conditions.csv")) and filename.startswith("S"): 
        #print(filename)
        i = filename.split("_")[1].split(".")[0]
        split = filename.split("bone")[0].split("click")[0].split("raw")[0]
        x = split.split("S")[1].split("U")
        x.append(i)
        lst.append(x)
    else:
        continue
dfData = pd.DataFrame(lst, columns=["S", "U", "I"]).drop_duplicates()
for c in dfData.columns:
    dfData[c] = pd.to_numeric(dfData[c])
print(len(dfData.S.unique()))
dfData = dfData.sort_values(["S", "U"])
dfData 

14


,S,U,I
24,1,1,0
25,1,2,0
49,2,3,0
59,2,4,0
39,4,5,0
15,4,6,0
41,5,7,0
18,5,8,0
0,6,9,0
35,6,10,0


In [33]:
def readRawData (pS, pId, i):
    #S10U17rawData_0.csv
    file = "%sS%iU%irawData_%i.csv" % (DATA_PATH, pS, pId, i)
    print(datetime.now(), file)
    with open(file, 'r') as f:
        first_line = f.readline()
    startTime = (int)(first_line.split(" ")[-1].replace("\n", ""))
    print(datetime.now(), file, "Start Time: %i" %startTime)
    df = pd.read_csv(file, skiprows=1, sep=";", low_memory=False)
    df.rename(columns={'PlayerSelect': 'IsPointer', 'UId': 'PId'}, inplace=True)
    x = len(df)
    if df.PId.dtype is np.array([object()]).dtype:
        df = df[(df.PId != "PId") & ( df.PId != "UId")]
        if (len(df) != x):
            print("Dropped %i header lines in file" % (x-len(df)))
    df = df[df['Timestamp'].isna() == False]
    for c in df.columns[:2]:
        df[c] = pd.to_numeric(df[c])
    for c in df.columns[3:]:
        if (isinstance(df[c].iloc[0], str)):
            df[c] = df[c].apply(lambda x: ast.literal_eval(x))
    df["StartTimeUnix"] = startTime
    df.columns = df.columns.str.replace("_", "")
    df.rename(columns={'Timestamp': 'TimeUnix', "HMDrot": "HMDRot"}, inplace=True)
    df["Time"] = pd.to_datetime(df.TimeUnix, unit='ms')
    
    df["SessionId"] = pS
    df["File"] = file
    print(datetime.now(), file, "Length: %i" % len(df))
    save_path = "%sdata_raw_S%i_U%i_v01.pkl" % (OUTPUT_PATH, pS, pId)
    df.to_pickle(save_path)
    return

In [29]:
%%time
df = readRawData(1,1,0)

2020-03-26 03:36:43.953324 ./data/S1U1rawData_0.csv
2020-03-26 03:36:43.954161 ./data/S1U1rawData_0.csv Start Time: 1519390556225
2020-03-26 03:38:49.293447 ./data/S1U1rawData_0.csv Length: 199933
CPU times: user 2min 6s, sys: 3.27 s, total: 2min 9s
Wall time: 2min 9s


In [34]:
%%time
## in case you are running our of memory replace parallel_apply with apply.
dfData.apply(lambda e: readRawData(e.S, e.U, e.I), axis=1)

2020-03-26 03:42:51.167403 ./data/S10U18rawData_0.csv
2020-03-26 03:42:51.167396 ./data/S12U22rawData_0.csv
2020-03-26 03:42:51.171981 ./data/S10U18rawData_0.csv Start Time: 1519832666106
2020-03-26 03:42:51.172489 ./data/S12U22rawData_0.csv Start Time: 1519909893139
2020-03-26 03:42:51.170323 ./data/S11U19rawData_0.csv
2020-03-26 03:42:51.170285 ./data/S1U2rawData_0.csv
2020-03-26 03:42:51.169632 ./data/S11U20rawData_0.csv
2020-03-26 03:42:51.170576 ./data/S15U28rawData_0.csv
2020-03-26 03:42:51.170542 ./data/S6U9rawData_0.csv
2020-03-26 03:42:51.170393 ./data/S1U1rawData_0.csv
2020-03-26 03:42:51.170329 ./data/S2U4rawData_0.csv
2020-03-26 03:42:51.171419 ./data/S14U25rawData_0.csv
2020-03-26 03:42:51.170432 ./data/S9U15rawData_0.csv
2020-03-26 03:42:51.171496 ./data/S7U11rawData_0.csv
2020-03-26 03:42:51.170113 ./data/S8U14rawData_0.csv
2020-03-26 03:42:51.170326 ./data/S13U23rawData_0.csv
2020-03-26 03:42:51.168769 ./data/S12U21rawData_0.csv
2020-03-26 03:42:51.175604 ./data/S5U7raw

KeyboardInterrupt: 

In [43]:
%%time
def readFile(e):
    save_path = "%sdata_raw_S%i_U%i_v01.pkl" % (OUTPUT_PATH, e.S, e.U)
    print(datetime.now(), save_path)
    return pd.read_pickle(save_path)

dfs = dfData[(dfData.U % 2) == 0].apply(lambda e: readFile(e), axis=1)
print(datetime.now(), "Done loading...")
df = pd.concat(dfs.to_list())
df.IsPointer = df.IsPointer.astype(bool)
df = df.sort_values("TimeUnix")
save_path = "%sdata_raw_Observer_v01.pkl" % (OUTPUT_PATH)
print(save_path)
df.to_pickle(save_path)

2020-03-26 03:59:02.608818 ./out/data_raw_S8_U14_v01.pkl
2020-03-26 03:59:05.778006 ./out/data_raw_S8_U14_v01.pkl
2020-03-26 03:59:06.056692 ./out/data_raw_S15_U28_v01.pkl
2020-03-26 03:59:06.262106 ./out/data_raw_S11_U20_v01.pkl
2020-03-26 03:59:06.413679 ./out/data_raw_S10_U18_v01.pkl
2020-03-26 03:59:06.892554 ./out/data_raw_S13_U24_v01.pkl
2020-03-26 03:59:07.250194 ./out/data_raw_S4_U6_v01.pkl
2020-03-26 03:59:07.589062 ./out/data_raw_S14_U26_v01.pkl
2020-03-26 03:59:07.832275 ./out/data_raw_S5_U8_v01.pkl
2020-03-26 03:59:08.053815 ./out/data_raw_S12_U22_v01.pkl
2020-03-26 03:59:08.370156 ./out/data_raw_S1_U2_v01.pkl
2020-03-26 03:59:08.530303 ./out/data_raw_S7_U12_v01.pkl
2020-03-26 03:59:08.744428 ./out/data_raw_S6_U10_v01.pkl
2020-03-26 03:59:09.034744 ./out/data_raw_S9_U16_v01.pkl
2020-03-26 03:59:09.254921 ./out/data_raw_S2_U4_v01.pkl
2020-03-26 03:59:09.441061 Done loading...
./out/data_raw_Observer_v01.pkl
CPU times: user 13.1 s, sys: 4.42 s, total: 17.6 s
Wall time: 17.5 s

In [55]:
%%time
def readFile(e):
    file = "%sdata_raw_S%i_U%i_v01.pkl" % (OUTPUT_PATH, e.S, e.U)
    print(datetime.now(), file)
    return pd.read_pickle(file)

dfs = dfData[(dfData.U % 2) == 1].apply(lambda e: readFile(e), axis=1)
print(len(dfs))
df = pd.concat(dfs.to_list())
df.IsPointer = df.IsPointer.astype(bool)
df = df.sort_values("TimeUnix")
save_path = "%sdata_raw_Pointer_v01.pkl" % (OUTPUT_PATH)
print(save_path)
df.to_pickle(save_path)

2020-03-26 04:05:32.999495 ./out/data_raw_S1_U1_v01.pkl
2020-03-26 04:05:41.078500 ./out/data_raw_S1_U1_v01.pkl
2020-03-26 04:05:43.485704 ./out/data_raw_S2_U3_v01.pkl
2020-03-26 04:05:46.144450 ./out/data_raw_S4_U5_v01.pkl
2020-03-26 04:05:50.602626 ./out/data_raw_S5_U7_v01.pkl
2020-03-26 04:05:54.079140 ./out/data_raw_S6_U9_v01.pkl
2020-03-26 04:05:58.386351 ./out/data_raw_S7_U11_v01.pkl
2020-03-26 04:06:01.825974 ./out/data_raw_S8_U13_v01.pkl
2020-03-26 04:06:05.097121 ./out/data_raw_S9_U15_v01.pkl
2020-03-26 04:06:08.393877 ./out/data_raw_S10_U17_v01.pkl
2020-03-26 04:06:15.978347 ./out/data_raw_S11_U19_v01.pkl
2020-03-26 04:06:18.313955 ./out/data_raw_S12_U21_v01.pkl
2020-03-26 04:06:22.859128 ./out/data_raw_S13_U23_v01.pkl
2020-03-26 04:06:43.935242 ./out/data_raw_S14_U25_v01.pkl
2020-03-26 04:07:13.237359 ./out/data_raw_S15_U27_v01.pkl
14
./out/data_raw_Pointer_v01.pkl
CPU times: user 3min 19s, sys: 1min 39s, total: 4min 59s
Wall time: 10min 15s
